# Bagging Explanation

This notebook is focused on the explanation of the Random Forest model used to perform the `Task 4: classification`.

## Preparation of the model

In [1]:
from os import path
import pandas as pd
from preprocessing import get_train_test_data

X_train, y_train, X_test, y_test, columns_to_keep = get_train_test_data()

In [3]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(
    class_weight = 'balanced',
    criterion = 'entropy',
    max_depth = 30,
    max_features = 4,
    min_samples_leaf = 3,
    min_samples_split = 20,
    n_estimators = 200
)

rfc.fit(X_train, y_train)

KeyboardInterrupt: 

# Feature importance